In [1]:
from google.colab import files
import pandas as pd
import os, re

# 1) Upload (may create e.g. "prepared_ridership_data (2).csv")
uploaded = files.upload()

# 2) Clean names and rename files back to the four you care about
target_names = {
    'ridership_data':       'ridership_data.xlsx',
    'station_lookup':       'station_lookup.csv',
    'prepared_ridership_data': 'prepared_ridership_data.csv',
    'pitt_customers':       'pitt_customers.csv'
}

for fname in uploaded.keys():
    # strip off any " (n)" before the extension
    clean = re.sub(r'\s\(\d+\)(?=\.)', '', fname)
    # if it's one of our targets (by base name), rename it
    base = os.path.splitext(clean)[0]
    if base in target_names:
        final_name = target_names[base]
        os.rename(fname, final_name)
        print(f"Renamed {fname} → {final_name}")
    else:
        # still rename stray duplicates to their clean name
        if fname != clean:
            os.rename(fname, clean)
            print(f"Renamed {fname} → {clean}")

# 3) Load each into a DataFrame
#    (will error if a file wasn’t uploaded—wrap in exists check if needed)
ridership_data_df = pd.read_excel('ridership_data.xlsx')
station_lookup_df = pd.read_csv('station_lookup.csv')
prepared_ridership_data_df = pd.read_csv('prepared_ridership_data.csv')
pitt_customers_df = pd.read_csv('pitt_customers.csv')

print("DataFrames loaded:")
print(" • ridership_data_df:", ridership_data_df.shape)
print(" • station_lookup_df:", station_lookup_df.shape)
print(" • prepared_ridership_data_df:", prepared_ridership_data_df.shape)
print(" • pitt_customers_df:", pitt_customers_df.shape)

Saving pitt_customers.csv to pitt_customers (1).csv
Saving prepared_ridership_data.csv to prepared_ridership_data (1).csv
Saving ridership_data.xlsx to ridership_data (1).xlsx
Saving station_lookup.csv to station_lookup (1).csv
Renamed pitt_customers (1).csv → pitt_customers.csv
Renamed prepared_ridership_data (1).csv → prepared_ridership_data.csv
Renamed ridership_data (1).xlsx → ridership_data.xlsx
Renamed station_lookup (1).csv → station_lookup.csv
DataFrames loaded:
 • ridership_data_df: (733790, 17)
 • station_lookup_df: (60, 6)
 • prepared_ridership_data_df: (651581, 24)
 • pitt_customers_df: (11858, 33)


In [2]:
# generate_route_map.py (Prepares clean CSV for use in Streamlit)

import pandas as pd

# === Load and normalize station lookup first ===
station_lookup = pd.read_csv("station_lookup.csv", encoding="utf-8-sig")
station_lookup.columns = station_lookup.columns.str.strip().str.lower().str.replace(" ", "_")

# === Load remaining data ===
ridership = pd.read_excel("ridership_data.xlsx", sheet_name="All trips data")
pitt_customers = pd.read_csv("pitt_customers.csv")

# === Clean and normalize ===
ridership.columns = ridership.columns.str.strip().str.lower().str.replace(" ", "_")
pitt_customers.columns = pitt_customers.columns.str.strip().str.lower().str.replace(" ", "_")

# === Convert mixed-format dates ===
def convert_excel_or_datetime(series):
    def parse(val):
        try:
            val = float(val)
            return pd.to_datetime(val, origin="1899-12-30", unit="d")
        except:
            return pd.to_datetime(val, errors="coerce")
    return series.apply(parse)

ridership["start_date"] = convert_excel_or_datetime(ridership["start_date"])
ridership["end_date"] = convert_excel_or_datetime(ridership["end_date"])

# Optional: print how many rows failed to parse
invalid_dates = ridership["start_date"].isna().sum()
print(f"❗ Unparsed start_date rows: {invalid_dates}")

# Proceed with year/month extraction
ridership["month"] = ridership["start_date"].dt.month
ridership["year"] = ridership["start_date"].dt.year

# === Tag Pitt Riders ===
ridership["is_pitt_rider"] = ridership["rider_id"].isin(pitt_customers["customer_id"])

# === Merge in lat/lon ===
ridership = ridership.merge(
    station_lookup[["name", "latitude", "longitude"]].rename(columns={
        "name": "start_station_name",
        "latitude": "start_lat",
        "longitude": "start_lon"
    }),
    on="start_station_name",
    how="left"
)

ridership = ridership.merge(
    station_lookup[["name", "latitude", "longitude"]].rename(columns={
        "name": "end_station_name",
        "latitude": "end_lat",
        "longitude": "end_lon"
    }),
    on="end_station_name",
    how="left",
    suffixes=("", "_end")
)

# === Drop rows with missing coordinates ===
ridership.dropna(subset=["start_lat", "start_lon", "end_lat", "end_lon"], inplace=True)

# === Save full enriched dataset for Streamlit ===
ridership.to_csv("prepared_ridership_data.csv", index=False)
print("✅ Preprocessed data saved to 'prepared_ridership_data.csv'")


❗ Unparsed start_date rows: 0
✅ Preprocessed data saved to 'prepared_ridership_data.csv'


In [4]:
# 3) Write out your Streamlit app to app.py
app_code = """
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium
from folium.plugins import HeatMap, MarkerCluster

st.set_page_config(page_title='POGOH Dashboard', layout='wide')
st.title('📍 POGOH Ridership Route Explorer')

@st.cache_data
def load_data():
    df = pd.read_csv('prepared_ridership_data.csv')
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r'[\\s\\(\\)\\-]+', '_', regex=True)
          .str.replace(r'[^a-z0-9_]', '', regex=True)
    )
    return df

df = load_data()

# Sidebar filters
with st.sidebar:
    st.header('🔍 Filters')
    start_station = st.selectbox('Start Station',
        ['All'] + sorted(df['start_station_name'].dropna().unique()))
    end_station = st.selectbox('End Station',
        ['All'] + sorted(df['end_station_name'].dropna().unique()))
    start_neighborhood = st.selectbox('Start Station Neighborhood',
        ['All'] + sorted(df['start_station_neighborhood'].dropna().unique()))
    end_neighborhood = st.selectbox('End Station Neighborhood',
        ['All'] + sorted(df['end_station_neighborhood'].dropna().unique()))
    membership_types = st.multiselect('Membership Types',
        sorted(df['product_name'].dropna().unique()))
    selected_years = st.multiselect('Year(s)',
        sorted(df['year'].dropna().unique()))
    selected_months = st.multiselect('Month(s)',
        sorted(df['month'].dropna().unique()))
    pitt_filter = st.radio('Include Pitt Riders',
        ['All','Only Pitt Riders','Exclude Pitt Riders'])
    if st.button('Clear Filters'):
        st.experimental_rerun()

# Apply filters
filtered = df.copy()
if start_station != 'All':
    filtered = filtered[filtered['start_station_name']==start_station]
if end_station != 'All':
    filtered = filtered[filtered['end_station_name']==end_station]
if start_neighborhood != 'All':
    filtered = filtered[filtered['start_station_neighborhood']==start_neighborhood]
if end_neighborhood != 'All':
    filtered = filtered[filtered['end_station_neighborhood']==end_neighborhood]
if membership_types:
    filtered = filtered[filtered['product_name'].isin(membership_types)]
if selected_years:
    filtered = filtered[filtered['year'].isin(selected_years)]
if selected_months:
    filtered = filtered[filtered['month'].isin(selected_months)]
if pitt_filter == 'Only Pitt Riders':
    filtered = filtered[filtered['is_pitt_rider']==True]
elif pitt_filter == 'Exclude Pitt Riders':
    filtered = filtered[filtered['is_pitt_rider']==False]

# Metrics
col1, col2 = st.columns(2)
with col1:
    st.metric("Total Rides", f"{df.shape[0]:,}")
with col2:
    st.metric("Filtered Rides", f"{filtered.shape[0]:,}")

# Build detail_table
breakdown_cols = ['start_station_name','end_station_name','product_name','is_pitt_rider']
if selected_months and len(selected_months)>1:
    filtered['period'] = filtered['month'].astype(str).str.zfill(2) + '-' + filtered['year'].astype(str)
    group_on = ['period']
else:
    group_on = ['year']

if not filtered.empty:
    pivot = filtered.groupby(breakdown_cols+group_on).agg(ride_count=('rider_id','count')).reset_index()
    detail_table = pivot.pivot_table(
        index=breakdown_cols,
        columns=group_on,
        values='ride_count',
        fill_value=0
    ).reset_index()
else:
    detail_table = pd.DataFrame()

# Map
st.subheader('🗺️ Route Map')
if not filtered.empty:
    mid_lat, mid_lon = filtered['start_lat'].mean(), filtered['start_lon'].mean()
else:
    mid_lat, mid_lon = 40.4406, -79.9959

m = folium.Map(location=[mid_lat, mid_lon], zoom_start=13)
route_counts = filtered.groupby(
    ['start_station_name','end_station_name','start_lat','start_lon','end_lat','end_lon']
).size().reset_index(name='count').sort_values('count', ascending=False).head(200)
unique_stations = {}

for _, row in route_counts.iterrows():
    # draw route lines
    folium.PolyLine(
        locations=[[row['start_lat'],row['start_lon']],[row['end_lat'],row['end_lon']]],
        color='blue', weight=1+4*(row['count']-route_counts['count'].min())/(route_counts['count'].max()-route_counts['count'].min()),
        opacity=0.6,
        tooltip=f"{row['start_station_name']} ➝ {row['end_station_name']} ({row['count']} trips)"
    ).add_to(m)
    unique_stations[row['start_station_name']] = (row['start_lat'],row['start_lon'])
    unique_stations[row['end_station_name']] = (row['end_lat'],row['end_lon'])

marker_cluster = MarkerCluster().add_to(m)
for name,(lat,lon) in unique_stations.items():
    st_count = filtered[filtered['start_station_name']==name].shape[0]
    en_count = filtered[filtered['end_station_name']==name].shape[0]
    folium.Marker([lat,lon],
        popup=f"<b>{name}</b><br>Starts: {st_count}<br>Ends: {en_count}",
        tooltip=name,
        icon=folium.Icon(icon='bicycle', prefix='fa', color='green')
    ).add_to(marker_cluster)

st_folium(m, width=1000, height=600)

# Table
st.subheader('📊 Detailed Ride Breakdown')
if detail_table.empty:
    st.write("No rides match your filter selection.")
else:
    last_col = detail_table.columns[-1]
    st.dataframe(detail_table.sort_values(by=last_col, ascending=False).reset_index(drop=True))
"""
with open("app.py","w") as f:
    f.write(app_code)

# 4) Kill any existing processes & ngrok tunnels
!kill $(pgrep -f streamlit) 2>/dev/null || echo "no streamlit"
!kill $(pgrep -f ngrok)     2>/dev/null || echo "no ngrok"

# 5) Authenticate ngrok
!ngrok authtoken 2vjVH7eZrBlTBkuOUHezijeuHG1_5YK725Fr3F5hkdWPyC4UB

# 6) Launch Streamlit + open ngrok tunnel
import subprocess, sys, time
from pyngrok import ngrok

# start Streamlit
subprocess.Popen([
    sys.executable, "-m", "streamlit", "run", "app.py",
    "--server.port", "8501", "--server.headless", "true"
])
time.sleep(5)

# open tunnel
public_url = ngrok.connect(8501).public_url
print("🚀 Your POGOH Dashboard is live at:", public_url)

^C
^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Your POGOH Dashboard is live at: https://2844-34-16-237-113.ngrok-free.app
